In [ ]:
import numpy as np
import pandas as pd

In [ ]:
data=pd.read_csv("train.csv")
data.head(n=10)

In [ ]:
data.info()

In [ ]:
columns_to_drop=["PassengerId","Embarked","Name","Ticket","Cabin"]

In [ ]:
data_clean=data.drop(columns_to_drop,axis=1)

In [ ]:
data_clean

In [ ]:
from sklearn.preprocessing import LabelEncoder

In [ ]:
le=LabelEncoder()

In [ ]:
data_clean["Sex"]=le.fit_transform(data_clean["Sex"])

In [ ]:
data_clean.info()

In [ ]:
data_clean=data_clean.fillna(data_clean["Age"].mean())

In [ ]:
data_clean.info()

In [ ]:
input_cols=["Pclass","Sex","Age","SibSp","Parch","Fare"]

In [ ]:
output_cols=["Survived"]

In [ ]:
X=data_clean[input_cols]

In [ ]:
Y=data_clean[output_cols]

In [ ]:
print(X.shape)

In [ ]:
def entropy(col):
    counts=np.unique(col,return_counts=True)
#     print(counts)
    N=float(col.shape[0])
    ent=0.0
    for i in counts[1]:
        p=i/N
        ent+=(-1.0*p*np.log2(p))
        
    return ent;
     

In [ ]:
col=np.array([1,1,1,1,0,1,0])

In [ ]:
entropy(col)

In [ ]:
def divide_data(x_data,fkey,fval):
    x_right=pd.DataFrame([],columns=x_data.columns)
    x_left=pd.DataFrame([],columns=x_data.columns)   
#             x_left=pd.concat([x_left,x_data.loc[i]])
#             x_right=x_right.append(x_data.loc[i])   
#               
    
    for i in range(x_data.shape[0]):
        val=x_data[fkey].loc[i]
        
        if val>fval:
            x_right=pd.concat([x_right,x_data.loc[i]])  
        else:
            x_left=pd.concat([x_left,x_data.loc[i]])
            
            
                              
            
    return x_right,x_left;

In [ ]:
x_left,x_right=divide_data(data_clean[:10],"Sex",0.5)
print(x_right)

In [ ]:
def information_gain(x_data,fkey,fval):
    left,right=divide_data(x_data,fkey,fval)
    l=float(left.shape[0])/x_data.shape[0]
    r=float(right.shape[0])/x_data.shape[0]
    
    if left.shape[0]==0 or right.shape[0]==0:
        return -1000000
    i_gain=entropy(x_data.Survived)-(l*entropy(left.Survived)+r*entropy(right.Survived))
    return i_gain
    
    

In [ ]:
# for f in X.columns:
#     print(f)
#     print(information_gain(data_clean,f,data_clean[f].mean()))

In [ ]:
class DecisionTree:
    
    
    def __init__(self,depth=0,max_depth=5):
        self.left=None
        self.right=None
        self.fkey=None
        self.fval=None
        self.max_depth=max_depth
        self.depth=depth
        self.target=None
        
    def train(self,X_train):
        features=["Pclass","Sex","Age","SibSp","Parch","Fare"]     
        info_gains=[]
        for i in features:
            i_gain=information_gain(X_train,i,X_train[i].mean())
            info_gains.append(i_gain)
            
        self.fkey=features[np.argmax(info_gains)]
        self.fval=X_train[self.fkey].mean()
        print(self.fkey)
        
        
        data_left,data_right=divide_data(X_train,self.fkey,self.fval)
        data_left=data_left.reset_index(drop=True)
        data_right=data_right.reset_index(drop=True)
        
        if data_left.shape[0]==0 or data_right.shape[0]==0:
            if X_train.Survived.mean()>=0.5:
                self.target="Survive"
            else:
                self.target="Dead"
            return
        if(self.depth>=self.max_depth):
            if X_train.Survived.mean()>=0.5:
                self.target="Survive"
            else:
                self.target="Dead"
            return
        self.left=DecisionTree(depth=self.depth+1,max_depth=self.max_depth)
        self.left.train(data_left)
        
        self.right=DecisionTree(depth=self.depth+1,max_depth=self.max_depth)
        self.right.train(data_right)
        
        if X_train.Survived.mean()>=0.5:
            if X_train.Survived.mean()>=0.5:
                self.target="Survive"
            else:
                self.target="Dead"
            return
            
            
    def predict(self,test):
        if test[self.fkey]>self.fval:
            if self.right in None:
                return self.target
            return self.right.predict(test)
        else:
            if self.left is None:
                return self.target
            return self.left.predict(test)
    
    
        
    
        
    

In [ ]:
d=DecisionTree()

In [ ]:
d.train(data_clean)

In [ ]:
split=int(0.7*data_clean.shape[0])
train_data=data_clean[:split]
test_data=data_clean[split:]
test_data=test_data.reset_index(drop=True)
print(train_data.shape)

In [ ]:
dt=DecisionTree()

In [ ]:
dt.train(train_data)

In [ ]:
print(dt.fkey)
print(dt.fval)
print(dt.left.fkey)
print(dt.right.fkey)

    